In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import shapiro

In [2]:
dfQuickSort16 = pd.read_csv("FinalAnalysis/CleanData/QuickClean-PC16.csv")
dfMergeSort16 = pd.read_csv("FinalAnalysis/CleanData/MergeClean-PC16.csv")
dfCountingSort16 = pd.read_csv("FinalAnalysis/CleanData/CountingClean-PC16.csv")
dfBubbleSort16 = pd.read_csv("FinalAnalysis/CleanData/BubbleClean-PC16.csv")


In [3]:
merge = []
quick = []
count = []
bubble = []

for case in ["best", "worst", "random"]:
    merge.append(dfMergeSort16[dfMergeSort16["Input case"].str.startswith(case)])
    quick.append(dfQuickSort16[dfQuickSort16["Input case"].str.startswith(case)])
    bubble.append(dfBubbleSort16[dfBubbleSort16["Input case"].str.startswith(case)])
    count.append(dfCountingSort16[dfCountingSort16["Input case"].str.startswith(case)])



In [4]:
def shapiroWilks(df, sizes, col):
    data = []
    size = []
    pvalues =[]

    for i in sizes:
        if 0.05 < shapiro(df[df["Size"]==i][col]).pvalue:
            data.append(1)
        else:
            data.append(0)
        size.append(i)
        pvalues.append(shapiro(df[df["Size"]==i][col]).pvalue)

    return pvalues

In [5]:
def getSharp(df, col):
    data = []
    size = []
    pvalues = []
    s = [25000,50000,75000,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000]
    s2 = [25000,50000,75000,100000,200000,300000,400000,500000]
    
    # print(type(df[col]))
    long = max(df["Size"])> 500000
    # print(df)

    for i in s if long else s2:
        if 0.05 < shapiro(df[df["Size"]==i][col]).pvalue:
            data.append(1)
        else:
            data.append(0)
        size.append(i)
        # print(shapiro(df[df["Size"]==i][col]).pvalue)
        pvalues.append(shapiro(df[df["Size"]==i][col]).pvalue)
    # print(data)
    # print(pvalues)

    return pvalues

In [6]:
def countGreaterThan(list):
    count = 0
    for item in list:
        if item > 0.05:
            count += 1

    return count

In [7]:
def percentNormal(pvalues):
    return countGreaterThan(pvalues) / len(pvalues) * 100

In [8]:
list = [merge, quick, count, bubble]
cases = ["best", "worst", "random"]
names = ["merge", "quick", "count", "bubble"]

dfCPU = pd.DataFrame({"Algorithm": ["merge", "quick", "counting", "bubble"]})
dfTime = pd.DataFrame({"Algorithm": ["merge", "quick", "counting", "bubble"]})
dfDRAM = pd.DataFrame({"Algorithm": ["merge", "quick", "counting", "bubble"]})
algCount = 0
for case in range(len(cases)):
    cpu = []
    time = []
    dram = []
    for alg in range(len(list)):
        cpu.append(percentNormal( getSharp (list[alg][case], "CPU (J)")))
        time.append(percentNormal( getSharp (list[alg][case], "Time (ms)")))
        dram.append(percentNormal( getSharp (list[alg][case], "DRAM (J)")))

    dfCPU["CPU "+cases[case]+" %"] = cpu
    dfTime["Time "+cases[case]+" %"] = time
    dfDRAM["DRAM "+cases[case]+" %"] = dram
    algCount += 1

dfCPU

,Algorithm,CPU best %,CPU worst %,CPU random %
0,merge,0.000000,7.692308,0.0
1,quick,15.384615,12.500000,0.0
2,counting,0.000000,7.692308,0.0
3,bubble,0.000000,12.500000,0.0


In [9]:
dfTime

,Algorithm,Time best %,Time worst %,Time random %
0,merge,53.846154,38.461538,15.384615
1,quick,61.538462,12.500000,0.000000
2,counting,15.384615,46.153846,0.000000
3,bubble,12.500000,12.500000,12.500000


In [10]:
dfDRAM

,Algorithm,DRAM best %,DRAM worst %,DRAM random %
0,merge,0.000000,0.0,0.0
1,quick,46.153846,0.0,0.0
2,counting,7.692308,0.0,0.0
3,bubble,0.000000,0.0,0.0
